In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import Callback
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler

In [ ]:
# Load and preprocess the Wine dataset
data = load_wine()
X = data.data
y = data.target.reshape(-1, 1)

In [ ]:
# One-hot encode the target variable
encoder = OneHotEncoder(sparse_output=False)
y = encoder.fit_transform(y)

In [ ]:
# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [ ]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
# Build the DNN model
model = Sequential([
    tf.keras.Input(shape=(X_train.shape[1],)),  # Input shape according to number of features
    Dense(10, activation='relu'),  # Hidden layer with 10 neurons
    Dense(3, activation='softmax')  # Output layer with softmax for 3 classes
])

model.compile(optimizer=SGD(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
# Custom callback to log weights and gradients after each epoch
class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        weights = [layer.get_weights() for layer in self.model.layers if len(layer.get_weights()) > 0]
        print(f"Epoch {epoch + 1}")
        for idx, (w, b) in enumerate(weights):
            print(f"Weights of layer {idx}:\n{w}")
            print(f"Biases of layer {idx}:\n{b}")

        # Calculate gradients using a manual step
        with tf.GradientTape() as tape:
            predictions = self.model(X_train, training=True)
            loss = self.model.compiled_loss(y_train, predictions)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        print("Gradients:")
        for grad in gradients:
            print(grad.numpy())
        print("\n" + "="*50 + "\n")

In [ ]:
# Train the model with the custom callback
model.fit(X_train, y_train, epochs=50, verbose=0, callbacks=[CustomCallback()])

Epoch 1
Weights of layer 0:
[[-0.27528307  0.39651468 -0.3350839  -0.42995718  0.1006896   0.09692512
  -0.40758157 -0.06868931  0.13325204 -0.46116388]
 [ 0.17032494  0.3010525   0.22374569 -0.01543572 -0.37574378  0.40991804
  -0.2517941  -0.13674933 -0.36003968  0.16666412]
 [-0.36136532 -0.5077527  -0.23131447  0.36171022 -0.02977601  0.21855848
  -0.02469334  0.00092557  0.46637753 -0.04163403]
 [ 0.43944967  0.13751194 -0.16166343 -0.36635745 -0.08881909 -0.26368508
   0.26238337  0.12574682 -0.49777696  0.50958985]
 [ 0.30063617  0.36809382  0.4697393   0.4830618   0.23626645 -0.23878603
  -0.4739502  -0.05448005  0.3310619  -0.00635991]
 [ 0.429165    0.50312454  0.47991893 -0.16527    -0.1074591   0.2572226
   0.17901982  0.17147171  0.36580268 -0.5132102 ]
 [-0.31262067 -0.01386955 -0.04578608  0.05872091  0.49872974 -0.4658127
   0.23739184 -0.23141962 -0.35612175 -0.37152377]
 [-0.46052822  0.21503851  0.18867481  0.37946057  0.0432437  -0.3533088
  -0.05435897 -0.43943974 

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:607: UserWarning: `model.compiled_loss()` is deprecated. Instead, use `model.compute_loss(x, y, y_pred, sample_weight, training)`.
  warnings.warn(


Gradients:
[[-0.11970925  0.00667183 -0.04497869  0.06597482  0.00185688 -0.00692966
   0.051339   -0.09024228 -0.09791648  0.06748439]
 [-0.04167687  0.09116722 -0.1260922   0.01489363  0.00890038  0.08730706
   0.00379905  0.07293867 -0.02604212 -0.0454395 ]
 [-0.09683655  0.01231695 -0.04737911  0.02703098  0.0070866  -0.00767191
   0.02718318  0.00619283 -0.0655817   0.06150134]
 [ 0.00101574  0.05902958 -0.05761585 -0.00599121  0.00628443  0.04449213
  -0.02151762  0.11735728  0.0415291  -0.03839229]
 [ 0.0134521   0.00334658  0.00962976 -0.00131486  0.00074073 -0.00228583
   0.04343681  0.0221933   0.02162851  0.01700884]
 [ 0.02608936 -0.08400594  0.11693449 -0.00339491 -0.0051753  -0.09823562
  -0.01031143 -0.08325772 -0.04553892  0.07260241]
 [ 0.0102315  -0.11568726  0.13832481 -0.00613949 -0.00618602 -0.1197904
  -0.01334178 -0.12527226 -0.07609115  0.09095772]
 [-0.0780217   0.07432673 -0.09697714  0.01864356  0.00262242  0.03514822
   0.01180091  0.00537469 -0.02142768  0.

In [ ]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Loss: {loss:.4f}, Test Accuracy: {accuracy:.4f}")

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9525 - loss: 0.2711 
Test Loss: 0.2860, Test Accuracy: 0.9444
